# IBM Data Science Capstone Project

**Hello and Welcome!**

This Notebook will be used solely for IBM Data Science Capstone Project! I wish a great day and a lot of luck reviewing my results!
<br/>
Below you can find couple of my favourite jokes about being Data Scientist to lighten your mood:

***

>*The data science motto: If at first you don’t succeed; call it version 1.0*

***

>*Why should you take a data scientist with you into the jungle? <br/>- They can take care of Python problems*

***

In [16]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Assignment #2: Segmenting and Clustering Neighborhoods in Toronto

The main task is to explore and cluster neihborhoods in Toronto, Canada, using Foresquare API and K-means clustring algorithm.

**Part 1**<br/>
Scrape Neighborhood list from wikipedia.

In [2]:
# Importing scraping libraries
import requests
from bs4 import BeautifulSoup as bs4

In [3]:
# Defining url and requesting source HTML
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)

In [4]:
# Souping the html :D
soup = bs4(r.content,'html')

In [30]:
# Find table by class name and convert to DataFrame
table = soup.find_all('table',{'class':'wikitable sortable'})
neighbrhd_df = pd.read_html(str(table[0]))[0]
neighbrhd_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
